In [11]:
import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(
    settings=Settings(chroma_client_auth_provider="chromadb.auth.basic.BasicAuthClientProvider",chroma_client_auth_credentials="admin:password123"))
client.heartbeat()  # this should work with or without authentication - it is a public endpoint
client.get_or_create_collection("test_collection")  # this is a protected endpoint and requires authentication
client.list_collections()  # this is a protected endpoint and requires authentication

[Collection(name=test_collection)]

In [12]:
import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(
    settings=Settings(chroma_client_auth_provider="chromadb.auth.basic.BasicAuthClientProvider",chroma_client_auth_credentials="user1:password123"))
client.heartbeat()  # this should work with or without authentication - it is a public endpoint
client.get_or_create_collection("test_collection")  # this is a protected endpoint and requires authentication
client.list_collections()  # this is a protected endpoint and requires authentication

[Collection(name=test_collection)]

In [13]:
import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(
    settings=Settings(chroma_client_auth_provider="chromadb.auth.basic.BasicAuthClientProvider",chroma_client_auth_credentials="invalid_user:password123"))
client.heartbeat()  # this should work with or without authentication - it is a public endpoint

client.list_collections()  # this is a protected endpoint and requires authentication

AuthorizationError: Unauthorized